In [27]:
import tensorflow as tf
import numpy as np
print(tf.config.list_physical_devices('GPU'))
from sklearn.metrics import recall_score, f1_score
from keras import backend as K
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import mixed_precision
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from tensorflow.keras import layers


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [28]:
mixed_precision.set_global_policy('mixed_float16')

In [29]:


# Load the training set from a CSV file
X_train = pd.read_csv('../data/X_train_balanceado.csv')
y_train = pd.read_csv('../data/y_train_balanceado.csv')

# Load the test set from a CSV file
X_test = pd.read_csv('../data/X_test_balanceado.csv')
y_test = pd.read_csv('../data/y_test_balanceado.csv')

In [30]:


# Create a scaler object
scaler = StandardScaler()

# Fit the scaler to the training data
scaler.fit(X_train)

# Scale the training and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
X_train_scaled.shape

(36456, 777)

In [32]:
y_train = np.array(y_train.values)
y_test = np.array(y_test.values)

In [33]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [34]:
model_id = 'lstm_1'
# Define the batch size
batch_size = 64

# Define the number of epochs
epochs = 25

# Define the TensorBoard callback
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S") + "_" + model_id + f"_bs_{batch_size}_ep_{epochs}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Define the ModelCheckpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_' + model_id + '_{epoch:02d}-{val_accuracy:.2f}.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1)

# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=4)

# Define the LSTM model
model = tf.keras.Sequential([
    layers.Reshape((777, 1), input_shape=(777,)),
    layers.BatchNormalization(),
    layers.LSTM(128, activation='tanh', return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(64, activation='tanh'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

# Define the optimizer with a lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate==0.0001)

# Compile the model with the new optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(), f1_m])





2023-06-21 02:23:22.494783: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-21 02:23:22.495799: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-21 02:23:22.496605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [35]:
# Fit the model with the new settings and callbacks
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_data=(X_test_scaled, y_test), callbacks=[tensorboard_callback, checkpoint_callback, early_stopping_callback])

Epoch 1/25


2023-06-21 02:23:23.177046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-21 02:23:23.178223: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-21 02:23:23.179076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

570/570 [==============================] - ETA: 0s - loss: 0.6910 - accuracy: 0.5281 - recall_3: 0.5801 - f1_m: 0.5394

2023-06-21 02:23:46.414494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-21 02:23:46.415450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-21 02:23:46.416188: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.64301, saving model to model_lstm_1_01-0.64.h5
570/570 [==============================] - 30s 48ms/step - loss: 0.6910 - accuracy: 0.5281 - recall_3: 0.5801 - f1_m: 0.5394 - val_loss: 0.6722 - val_accuracy: 0.6430 - val_recall_3: 0.6891 - val_f1_m: 0.7587
Epoch 2/25
569/570 [============================>.] - ETA: 0s - loss: 0.6897 - accuracy: 0.5330 - recall_3: 0.5924 - f1_m: 0.5472
Epoch 2: val_accuracy improved from 0.64301 to 0.74109, saving model to model_lstm_1_02-0.74.h5
570/570 [==============================] - 27s 47ms/step - loss: 0.6897 - accuracy: 0.5331 - recall_3: 0.5928 - f1_m: 0.5475 - val_loss: 0.5458 - val_accuracy: 0.7411 - val_recall_3: 0.8286 - val_f1_m: 0.8364
Epoch 3/25
569/570 [============================>.] - ETA: 0s - loss: 0.6880 - accuracy: 0.5440 - recall_3: 0.5819 - f1_m: 0.5528
Epoch 3: val_accuracy did not improve from 0.74109
570/570 [==============================] - 27s 47ms/step - loss: 0.6880 - accurac